In [7]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
import numpy as np
import pandas as pd
import h5py
from tqdm import tqdm
import scipy.io as sio
from scipy import stats
from importlib import reload
from matplotlib import pyplot as plt
import sys
#sys.path.append('/home/jma819/post_cmfe_analysis')
sys.path.append('/Users/johnmarshall/Documents/Analysis/PythonAnalysisScripts/post_cmfe_analysis')
import python_utils_jjm as utils_jjm
import dlc_utils
import caiman
import statsmodels.formula.api as smf
import scipy.spatial.distance as dist
import itertools
import math
import warnings
import numbers
from multiprocessing import Pool
from functools import partial
warnings.filterwarnings(action='once')

/Users/johnmarshall/anaconda3/envs/caiman/lib/python3.6/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.decomposition.incremental_pca module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.decomposition. Anything that cannot be imported from sklearn.decomposition is now part of the private API.
  warnings.warn(message, FutureWarning)
/Users/johnmarshall/anaconda3/envs/caiman/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/johnmarshall/anaconda3/envs/caiman/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of 

In [1]:
import matlab.engine 
import os

In [2]:
eng = matlab.engine.start_matlab()

In [3]:
os.chdir('/Users/johnmarshall/Documents/Analysis/PythonAnalysisScripts/post_cmfe_analysis/openfield_analysis/spatial_clustering/jones_scripts')
eng.cd('/Users/johnmarshall/Documents/Analysis/PythonAnalysisScripts/post_cmfe_analysis/openfield_analysis/spatial_clustering/jones_scripts')

'/Users/johnmarshall/Documents/Analysis/PythonAnalysisScripts/post_cmfe_analysis/openfield_analysis/spatial_clustering'

In [5]:
## import Jones's functions
#eng.help(eng.getPaddedSignalPeaks())

In [8]:
reload(utils_jjm)
reload(dlc_utils)

<module 'dlc_utils' from '/Users/johnmarshall/Documents/Analysis/PythonAnalysisScripts/post_cmfe_analysis/dlc_utils.py'>

In [9]:
cnmfe_file_key = pd.read_csv('/volumes/My_Passport/dlc_analysis/behavcamvideos/cnmfe_file_key.csv')
#cnmfe_file_key = pd.read_csv('/projects/p30771/dlc_analysis/openfield_dlc_output/cnmfe_file_key.csv')

In [10]:
cnmfe_base_dir = '/Volumes/My_Passport/cnmfe_analysis_files/batch_output_files/'
#cnmfe_base_dir = '/projects/p30771/MATLAB/CNMF_E_jjm/quest_MATLAB_cnmfe/batch_output_files/'
wt_CNMFE_file = ['30-Mar_20_39_05_out.mat', '30-Mar_20_45_16_out.mat', '27-Feb_17_33_59_out.mat', '22-Mar_22_52_02_out.mat',
                 '28-Feb_16_10_05_out.mat', '27-Feb_17_32_15_out.mat', '28-Feb_16_21_21_out.mat', '25-Mar_13_27_27_out.mat',
                 '25-Mar_14_22_02_out.mat', '25-Mar_14_22_44_out.mat', '26-Mar_18_33_55_out.mat', '27-Mar_00_26_12_out.mat', '27-Mar_00_48_46_out.mat']

ko_CNMFE_files = ['31-Mar_13_28_15_out.mat', '29-Mar_21_42_20_out.mat', '13-Apr_17_57_40_out.mat', '29-Mar_14_27_55_out.mat', '13-Apr_16_01_20_out.mat',
                 '13-Apr_16_11_27_out.mat', '29-Mar_13_39_44_out.mat']

In [11]:
tracking_files = ['/Volumes/My_Passport/dlc_analysis/behavcamvideos/open_field_miniscope/'+utils_jjm.find_behavior_tracking(fname, cnmfe_file_key) for fname in wt_CNMFE_file] 
#tracking_files = ['/projects/p30771/dlc_analysis/openfield_dlc_output/'+utils_jjm.find_behavior_tracking(fname, cnmfe_file_key) for fname in wt_CNMFE_file] 

In [12]:
CNMFE_dir_paths_list = [str(cnmfe_base_dir+fname) for fname in wt_CNMFE_file]
#CNMFE_dir_paths_list

In [13]:
binning_time = 1 # seconds 
body_part_for_tracking = 'tail_base' #
number_of_bins = 50 #
polynomial_degree = 2 #

grouped_raw_data = {}
success = []
failed = []
for CNMFE_file, tracking_file in zip(CNMFE_dir_paths_list, tracking_files):
    #print(CNMFE_file)
    #print(tracking_file)
    try:
        # load cell fluorescence 
        cell_fluorescence = sio.loadmat(CNMFE_file)
        C_timedelta = utils_jjm.create_fluorescence_time_delta(cell_fluorescence['C'])
        C_normalized = C_timedelta.apply(utils_jjm.normalize).set_index(pd.to_timedelta(np.linspace(0, (len(C_timedelta)-1)*(1/20), len(C_timedelta)), unit='s'), drop=True)
        C_z_scored = C_timedelta.apply(stats.zscore).set_index(pd.to_timedelta(np.linspace(0, (len(C_timedelta)-1)*(1/20), len(C_timedelta)), unit='s'), drop=True)
        C_normalized_z_scored = C_normalized.apply(stats.zscore).set_index(pd.to_timedelta(np.linspace(0, (len(C_normalized)-1)*(1/20), len(C_normalized)), unit='s'), drop=True)
        # create tracking time deltas
        interpolated = utils_jjm.prepare_timedelta_dfs(tracking_file)
        #load spatial components by session
        com_df, spatial_components = utils_jjm.return_spatial_info(CNMFE_file, 0.6)
        cell_contours, for_dims = utils_jjm.create_contour_layouts(spatial_components)
        #C_z_scored_filtered = utils_jjm.filter_out_by_size(C_z_scored, cell_contours, for_dims, 0.6, 100)
        #store results 
        grouped_raw_data[tracking_file.split('/')[-2]] = {'C': C_timedelta, 'C_z_scored': C_z_scored, 'C_normalized': C_normalized, 'C_normalized_z_scored': C_normalized_z_scored, 
                                                          'interpolated' : interpolated, 'com' : com_df, 'spatial_components' : spatial_components, 'cell_contours': cell_contours,  
                                                         'for_dims' : for_dims}
        success.append((tracking_file.split('/')[-2], CNMFE_file.split('/')[-1]))
    except FileNotFoundError:
        failed.append(tracking_file)
    except OSError:
        failed.append(tracking_file)

## spatial clustering

In [14]:
## combine dfs for each session to bin velocity values across dfs
#downsample
new_sampling_interval = .2
V_df = pd.concat([grouped_raw_data[session]['interpolated'].resample(str(new_sampling_interval)+'S').max() 
                  for session in list(grouped_raw_data.keys())], keys=list(grouped_raw_data.keys()))
all_sessions_v_bins = pd.cut(V_df['tail_base'], bins=50)

V_df['velocity_bins'] = all_sessions_v_bins

In [75]:
# compile and filter fluorescence
filtered_for_analysis = {}
for session in list(grouped_raw_data.keys()):
    filtered_for_analysis[session] = utils_jjm.filter_out_by_size(grouped_raw_data[session]['C_normalized_z_scored'], grouped_raw_data[session]['cell_contours'], 
                                                                  grouped_raw_data[session]['for_dims'], 0.6, 100)
C_df = pd.concat([filtered_for_analysis[session].resample(str(new_sampling_interval)+'S').max()
                  for session in list(grouped_raw_data.keys())], keys=list(grouped_raw_data.keys()))
C_df['velocity_bins'] = all_sessions_v_bins
#if want to normalize bins by each indiv mouse 
#C_df['by_mouse_vbins'] = V_df['by_mouse_vbins']

In [16]:
# clustering info per session 
coactivity_dfs = {}
for session in tqdm(list(grouped_raw_data.keys())):
    # get indicies of small cells 
    cells_to_drop = np.array([cell for cell in range(1, len(grouped_raw_data[session]['cell_contours'])+1) if 
                              len(np.array(np.where(grouped_raw_data[session]['for_dims'][cell]>0.6)[0]))<100])
    #compile and filter cell centers of mass
    filtered_centers_of_mass = grouped_raw_data[session]['com'].drop(cells_to_drop, axis =0)
    coactivity_dfs[session] = filtered_centers_of_mass

100%|██████████| 13/13 [00:03<00:00,  4.27it/s]


In [17]:
com_df = pd.concat(coactivity_dfs)

In [18]:
## save outputs if needed 
#com_df.loc['GRIN026_H16_M35_S34'].to_csv('/Volumes/My_Passport/cnmfe_analysis_files/OpenFieldAnalysis/2020/GRIN026_H16_M35_S34_com.csv')
#com_df.to_csv('/Volumes/My_Passport/cnmfe_analysis_files/OpenFieldAnalysis/2020/wt_com.csv')

In [19]:
## start analysis with Jones's scripts

In [20]:
C_df.loc['GRIN026_H16_M35_S34'].head()

,1,2,3,4,6,7,8,9,10,11,...,164,165,166,167,168,169,170,121,139,152
00:00:00,-0.251028,0.226838,0.345420,0.555802,0.803742,1.306358,0.504697,0.930071,1.225453,-0.036438,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00:00:00.200000,-0.176719,0.232539,0.329185,0.351416,0.728380,1.111226,0.561055,0.878875,1.151151,-0.092282,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00:00:00.400000,0.087449,0.185831,0.348359,0.328111,0.676191,1.191656,0.708289,0.827758,1.109766,-0.070758,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00:00:00.600000,0.158850,0.239402,0.284616,0.220333,0.832546,1.024923,0.640687,0.780411,1.152683,-0.105533,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00:00:00.800000,0.041801,0.165466,0.350471,0.515427,0.747269,1.448043,0.791051,0.877808,1.022892,-0.133374,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#get fluorescence traces for one animal
cell_traces = C_df.loc['GRIN026_H16_M35_S34'].iloc[:,:].dropna(axis=1).values.transpose()

In [71]:
np.shape(cell_traces)[0]

76

In [28]:
#xy centroids
cell_XY_coords = com_df.loc['GRIN026_H16_M35_S34'].iloc[:,:].dropna(axis=1).values.transpose()

In [31]:
#convert to MATLAB array 
cell_traces_MATLAB = matlab.double(cell_traces.tolist())
cell_XY_coords_MATLAB = matlab.double(cell_XY_coords.tolist())

In [23]:
#output signal peaks 
signal_peaks = eng.computeSignalPeaks(cell_traces_MATLAB, 'doMovAvg', 0, 'reportMidpoint', 1, 'numStdsForThresh', 2.5)

In [25]:
#pad signal peaks
padded_signal_peaks = eng.getPaddedSignalPeaks(signal_peaks)

In [26]:
#eng.size(signal_peaks)
eng.size(padded_signal_peaks)

matlab.double([[76.0,6250.0]])

In [30]:
#get cell distances multiply by pixel to micron scale
cellDistances = eng.pdist(cell_XY_coords_MATLAB, 'euclidean')*2.5

(2, 75)

In [77]:
#get velocity bins
#change this to get indicies of regions when in velocity bin 
for session in ['GRIN026_H16_M35_S34']:
    binned_regions = [C_df.loc[session][C_df.loc[session]['velocity_bins']==v_bin].drop(['velocity_bins'], axis=1).mean(axis=0).dropna() for
                  v_bin in all_sessions_v_bins.values.categories.values]

In [83]:
C_df.loc[session][C_df.loc[session]['velocity_bins']==all_sessions_v_bins.values.categories.values[1]]

,1,2,3,4,6,7,8,9,10,11,...,165,166,167,168,169,170,121,139,152,velocity_bins
00:00:11.800000,-0.587434,-0.339512,-0.341468,0.006575,-0.316810,1.490397,1.142735,-0.174554,0.099518,-0.082213,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(1.179, 2.314]"
00:00:12,-0.589682,-0.388927,-0.398329,-0.196344,-0.369189,1.533771,1.020827,-0.159541,0.104046,-0.131382,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(1.179, 2.314]"
00:00:12.200000,-0.591585,-0.430463,-0.405565,-0.323407,-0.408156,1.396996,1.174801,-0.179829,0.093751,-0.185975,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(1.179, 2.314]"
00:00:12.400000,-0.593196,-0.465376,-0.429069,-0.071299,-0.437147,1.539698,0.982431,-0.180132,0.055734,-0.073265,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(1.179, 2.314]"
00:00:12.600000,-0.594558,-0.406529,-0.469242,-0.162280,-0.458714,2.254715,0.970112,-0.207398,-0.010703,-0.032992,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(1.179, 2.314]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
00:20:41.400000,1.624127,1.657836,0.342325,0.463371,-0.078184,0.858487,0.379236,0.630424,0.576608,0.266320,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(1.179, 2.314]"
00:20:41.600000,1.544836,1.702978,0.685873,0.212050,-0.116660,0.399792,0.840228,0.801795,0.541635,0.272197,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(1.179, 2.314]"
00:20:45,0.088219,0.208085,-0.389049,-0.421890,-0.415969,2.403615,3.609779,-0.082311,-0.205335,2.931422,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(1.179, 2.314]"
00:20:48,1.968105,3.443665,3.012007,1.394830,1.471494,-0.221139,-0.323749,1.255900,0.625339,0.718826,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(1.179, 2.314]"


In [81]:
#display the values for the bin ranges
all_sessions_v_bins.values.categories.values

<IntervalArray>
[(-0.0114, 1.179], (1.179, 2.314], (2.314, 3.448], (3.448, 4.582], (4.582, 5.716] ... (51.08, 52.214], (52.214, 53.348], (53.348, 54.482], (54.482, 55.616], (55.616, 56.751]]
Length: 50, closed: right, dtype: interval[float64]

In [48]:
#input frames to analyze
frames_to_analyze = matlab.double(np.linspace(0, np.shape(cell_traces)[1]-1, np.shape(cell_traces)[1]).tolist())
within_frames_to_analyze = matlab.double(np.linspace(0, np.shape(cell_traces)[1]-1, np.shape(cell_traces)[1]).tolist())

In [69]:
# cell distance bins
maxDist = matlab.double([500]) #compare out to 500 um
binSize = matlab.double([20]) #compare in 20 um bins
numBins = matlab.double([maxDist[0][0]/binSize[0][0]-1]) #throw out the first bin (overlapping cells)
binVector = matlab.double(np.linspace(int(binSize[0][0]), int(maxDist[0][0]), int(numBins[0][0])+1).tolist())

In [72]:
#other needed variables
numCells = matlab.double([np.shape(cell_traces)[0]])

In [ ]:
# get Jaccard scores and compare to shuffle

eng.p355_jaccard_shuffle(padded_signal_peaks, frames_to_analyze, within_frames_to_analyze, cell_XY_coords_MATLAB, numCells, cellDistances, numBins, binVector, 1)



















In [ ]:
# clustering info per session 
coactivity_dfs = {}
for session in tqdm(list(grouped_raw_data.keys())):
    # get indicies of small cells 
    cells_to_drop = np.array([cell for cell in range(1, len(grouped_raw_data[session]['cell_contours'])+1) if 
                              len(np.array(np.where(grouped_raw_data[session]['for_dims'][cell]>0.6)[0]))<100])
    #compile and filter cell centers of mass
    filtered_centers_of_mass = grouped_raw_data[session]['com'].drop(cells_to_drop, axis =0)

    com_distances = utils_jjm.get_pairwise_distance_by_session(filtered_centers_of_mass)

    ##get binned fluorescence and calc Jaccard scores
    #arguments are sample widths to bin and z score threshold
    cells_in_session = C_df.loc[session].dropna(axis=1).drop('msCamFrame', axis=1)
    binned_fluorescence = cells_in_session.apply(utils_jjm.binning_function_uncrop, args=[1, 2])

    reindexed = binned_fluorescence.set_index(int(x) for x in np.linspace(0, len(binned_fluorescence)-1, len(binned_fluorescence)))

    #ks_results_2sided, ks_one_sided_more, ks_one_sided_less, coactivity_dfs = utils_jjm.spatial_coordination_by_session(reindexed, 10, com_distances)
    coactivity_dfs[session] = utils_jjm.spatial_coordination_by_session(reindexed)
    
    

In [ ]:
#save coactivity info to h5py file
for session in tqdm(list(grouped_raw_data.keys())):
    f = h5py.File("/Users/johnmarshall/Documents/Analysis/PythonAnalysisScripts/post_cmfe_analysis/openfield_analysis/spatial_clustering/" + str(session) +"clustering.h5", "w")

    for time_pt in range(len(coactivity_dfs[session])):
        dset = f.create_dataset(str(time_pt), np.shape(coactivity_dfs[session][time_pt].values), dtype=np.float32)
        dset[:, :] = coactivity_dfs[session][time_pt].values

    cell_pair_ids = f.create_dataset('cell_ids', np.shape(list(coactivity_dfs[session][0].columns)), dtype='i')
    cell_pair_ids[:, :] = list(coactivity_dfs[session][0].columns)

    f.close()

In [ ]:
spatial_coordination_by_session = {}
for session in list(grouped_raw_data.keys()):
    print(session)
    f = h5py.File("/Users/johnmarshall/Documents/Analysis/PythonAnalysisScripts/post_cmfe_analysis/openfield_analysis/spatial_clustering/" + str(session) +"clustering.h5", "r")
# get indicies of small cells 
    cells_to_drop = np.array([cell for cell in range(1, len(grouped_raw_data[session]['cell_contours'])+1) if 
                          len(np.array(np.where(grouped_raw_data[session]['for_dims'][cell]>0.6)[0]))<100])
#compile and filter cell centers of mass
    filtered_centers_of_mass = grouped_raw_data[session]['com'].drop(cells_to_drop, axis =0)
    com_distances = utils_jjm.get_pairwise_distance_by_session(filtered_centers_of_mass)

# get one sided ks tests from h5 data set
    binnums = 10
    ks_one_sided_more = []
    ks_one_sided_less = []
    for time_point in time_points:
        active_cells = [cell_ids[x] for x in [index[0] for index in np.argwhere(f[str(time_points[time_point])][0, 0:len(cell_ids)])]]
        active_cell_distances = [com_distances[cell].values[0] for cell in active_cells]
        cum_results_coactive = stats.cumfreq(active_cell_distances, numbins=binnums, defaultreallimits=(0, 500))
        less_result = stats.kstest(cum_results_coactive.cumcount/len(active_cell_distances), 'norm', alternative='less')
        more_result = stats.kstest(cum_results_coactive.cumcount/len(active_cell_distances), 'norm', alternative='greater')
        ks_one_sided_more.append(more_result)
        ks_one_sided_less.append(less_result)
    
    spatial_coordination_by_session[session] = {'ks_one_sided_more' : ks_one_sided_more, 'ks_one_sided_less' : ks_one_sided_less}

In [ ]:
session = list(grouped_raw_data.keys())[2]

In [ ]:
f = h5py.File("/Users/johnmarshall/Documents/Analysis/PythonAnalysisScripts/post_cmfe_analysis/openfield_analysis/spatial_clustering/" + str(session) +"clustering.h5", "r")
# get indicies of small cells 
cells_to_drop = np.array([cell for cell in range(1, len(grouped_raw_data[session]['cell_contours'])+1) if 
                          len(np.array(np.where(grouped_raw_data[session]['for_dims'][cell]>0.6)[0]))<100])
#compile and filter cell centers of mass
filtered_centers_of_mass = grouped_raw_data[session]['com'].drop(cells_to_drop, axis =0)
com_distances = utils_jjm.get_pairwise_distance_by_session(filtered_centers_of_mass)

In [ ]:
cell_ids = f['cell_ids']

In [ ]:
time_point = 95

In [ ]:
len(cell_ids)

In [ ]:
cell_ids[10]

In [ ]:
np.argwhere(f[str(time_point)][0, 0:len(cell_ids)])

In [ ]:
active_cells = [cell_idx[0] for cell_idx in np.argwhere(f[str(time_point)][0, 0:len(cell_ids)])]

In [ ]:
len(filtered_centers_of_mass)

In [ ]:
[cell_ids[cell] for cell in active_cells]

In [ ]:
cell_ids[active_cells[0]][0]

In [ ]:
cell_ids[active_cells[0]][1]

In [ ]:
com_distances[5]

In [ ]:
[com_distances[cell_ids[cell_idx][0]][cell_ids[cell_idx][1]].values[0] for cell_idx in active_cells]

In [ ]:
for time_pt in range(6251):
    if len(np.argwhere(f[str(time_pt)][0, 0:len(cell_ids)])) > 1:
        print(time_pt)
        break 
    

In [ ]:
len(f.keys())

In [ ]:
# get one sided ks tests from h5 data set
binnums = 10
ks_one_sided_more = []
ks_one_sided_less = []
cell_ids = f['cell_ids']
for time_point in time_points:
    print(time_point)
    active_cells = [cell_idx[0] for cell_idx in np.argwhere(f[str(time_point)][0, 0:len(cell_ids)])]
    active_cell_distances = [com_distances[cell_ids[cell_idx][0]][cell_ids[cell_idx][1]].values[0] for cell_idx in active_cells]
    cum_results_coactive = stats.cumfreq(active_cell_distances, numbins=binnums, defaultreallimits=(0, 500))
    less_result = stats.kstest(cum_results_coactive.cumcount/len(active_cell_distances), 'norm', alternative='less')
    more_result = stats.kstest(cum_results_coactive.cumcount/len(active_cell_distances), 'norm', alternative='greater')
    ks_one_sided_more.append(more_result)
    ks_one_sided_less.append(less_result)
    
spatial_coordination_by_session[session] = {'ks_one_sided_more' : ks_one_sided_more, 'ks_one_sided_less' : ks_one_sided_less}

In [ ]:
coord_index = np.nan_to_num([math.log(result.pvalue, 10)*-1 for result in ks_one_sided_less])

In [ ]:
plt.plot(coord_index[1700:2500])
plt.show()

In [ ]:
plt.plot(V_df.loc[session]['tail_base'].values[1700:2500])
plt.show()

In [ ]:
## for plotting average spatial coordination during trigger regions

In [ ]:

triggered_activity_across_sessions = {}
for session in list(grouped_raw_data.keys()):
    #inputs activity_threshold, resting_threshold, resting_baseline 
    crossing_indicies = utils_jjm.select_trigger_regions(binned_velocity_df[session], 0.5, 0.5, 20)
    #inputs time_to_plot
    threshold_activity = utils_jjm.average_triggered_regions(V_df.loc[session]['tail_base'].values, crossing_indicies, 40)
    
    f_threshold_activity = utils_jjm.average_triggered_regions(spatial_coordination_activity_in_session 
            , crossing_indicies, 40)
    triggered_activity_across_sessions[session] = pd.concat([threshold_activity, f_threshold_activity], axis=1, keys=['velocity', 'spatial_coordination'])

In [ ]:
#triggered_activity_across_sessions

In [ ]:
#concat by mouse 
concacted_by_mouse = {}
for mouse in list(set([session[0:7] for session in list(triggered_activity_across_sessions.keys())])):
    dfs_by_mouse = []
    sessions = []
    for session in list(triggered_activity_across_sessions.keys()):
        if mouse in session:
            if not(triggered_activity_across_sessions[session].empty):
                means = triggered_activity_across_sessions[session].mean(axis=1, level=0)
                dfs_by_mouse.append(means)
                sessions.append(session)
    if len(dfs_by_mouse)>0:
        concacted_by_mouse[mouse] = pd.concat(dfs_by_mouse, axis=1, keys=sessions)
combined_by_mouse = pd.concat(list(concacted_by_mouse.values()), axis=1, keys=list(concacted_by_mouse.keys()))

combined_by_mouse.head()

In [ ]:
binnums = 10
#ks_results_2sided = []
ks_one_sided_more = []
ks_one_sided_less = []
for time_point in tqdm(time_points):
    active_cells = [cell_ids[x] for x in [index[0] for index in np.argwhere(f[str(time_points[time_point])][0, 0:len(cell_ids)])]]
    coactive_cell_distances = [com_distances[cell].values[0] for cell in active_cells]
    #non_coactive_distances = com_distances[coactivity_df[coactivity_df == 0].dropna(axis=1).columns]
    #linear_distribution = np.linspace(1, len(coactive_cell_distances.columns), binnums)
    #binnums=10
    cum_results_coactive = stats.cumfreq(coactive_cell_distances, numbins=binnums, defaultreallimits=(0, 500))
    #cum_results_non_coactive = stats.cumfreq(non_coactive_distances.values[0], numbins=binnums, defaultreallimits=(0, 500))
    #plt.plot(np.linspace(0, 500, binnums), cum_results_coactive.cumcount/len(coactive_cell_distances.values[0]))
    #plt.plot(np.linspace(0, 500, binnums), cum_results_non_coactive.cumcount/len(non_coactive_distances.values[0]))
    #plt.plot(np.linspace(0, 500, binnums), linear_distribution/len(coactive_cell_distances.values[0]))
    #plt.show()
    less_result = stats.kstest(cum_results_coactive.cumcount/len(coactive_cell_distances), 'norm', alternative='less')
    more_result = stats.kstest(cum_results_coactive.cumcount/len(coactive_cell_distances), 'norm', alternative='greater')
    ks_one_sided_more.append(more_result)
    ks_one_sided_less.append(less_result)
    #coordination_index = utils_jjm.create_coordination_index(more_result, less_result)
    #two sided test between coactive and noncoactive distribution 
    #ks_result = stats.ks_2samp(cum_results_coactive.cumcount/len(coactive_cell_distances.values[0]), cum_results_non_coactive.cumcount/len(non_coactive_distances.values[0]))
    #coordination_indicies.append(coordination_index)
    #ks_results_2sided.append(ks_result)
    

In [ ]:
x = np.linspace(-40*.2, 40*.2, 80)
mean=combined_by_mouse.mean(axis=1, level=2)['velocity'].values
plt.plot(x, combined_by_mouse.mean(axis=1, level=2)['velocity'].values, color='k')
std_error = (combined_by_mouse.std(axis=1, level=2)['velocity'])/math.sqrt(combined_by_mouse.mean(axis=1, level=0).shape[1])
plt.fill_between(x, mean-std_error, mean+std_error)
ax = plt.gca()
ax.axvline(x=(0), linestyle='--', color='r')
plt.show()

In [ ]:
x = np.linspace(-40*.2, 40*.2, 80)
mean = combined_by_mouse.mean(axis=1, level=2)['fluorescence'].values
plt.plot(x, mean, color='k')
std_error = (combined_by_mouse.std(axis=1, level=2)['fluorescence'])/math.sqrt(combined_by_mouse.mean(axis=1, level=0).shape[1])
plt.fill_between(x, mean-std_error, mean+std_error)
ax = plt.gca()
ax.axvline(x=0, linestyle='--', color='r')
plt.show()